# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

Rappel : pensez à tester toutes vos fonctions sur des exemples.

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
2. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
3. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + 2$$
4. Définir une fonction récursive `v` telle que `v a n` renvoie $v_n$ définie par (on fera attention à n'utiliser qu'un appel récursif) :
$$v_0 = a$$
$$v_{n + 1} = \frac{1}{2}(v_n + \frac{a}{v_n})$$
5. Calculer `v a n` pour plusieurs valeurs de `a` et de `n` : que peut-on conjecturer sur la limite de $v_n$ ? Optionnel : le démontrer.  
6. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [6]:
let divise a b =
    a/b != 0


val divise : int -> int -> bool = <fun>


In [8]:
divise 2 3

- : bool = false


In [30]:
let rec somme n =
    if n = 1
    then 1
    else n*n + somme (n-1)

val somme : int -> int = <fun>


In [31]:
somme 3

- : int = 14


In [40]:
let rec u n =
    if n = 0.
    then 42.
    else 3.*. u(n-.1.)**0.5 +.2.

val u : float -> float = <fun>


In [46]:
u 1.

- : float = 21.4422220952235811


In [21]:
let rec v a n =
    if n = a
    then a
    else let x = v a (n -.1.) in
         0.5 *. (x +. (a/. x));

val v : float -> float -> float = <fun>


In [52]:
v 2. 5.

- : float = 1.41421568627450966


5. Il semblerait que lim (Vn) = sqrt (a)

In [39]:
let rec euclide a b =
    if b = 0
    then a
    else let r = a mod b in
         euclide b r;

val euclide : int -> int -> int = <fun>


In [41]:
euclide 30 10

- : int = 10


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Attention** : pour mettre au carré, utiliser une variable plutôt que faire 2 appels récursifs, ce qui augmenterait beaucoup le nombre d'opérations.  
**Remarque** : On montrera plus tard que cette 2ème version demande de l'ordre de $\ln(n)$ multiplications seulement.

In [71]:
let rec puissance a n =
    if n = 0
    then 1
    else a * puissance a (n-1)

val puissance : int -> int -> int = <fun>


In [73]:
puissance 2 2

- : int = 4


In [107]:
let rec exp_rapide a n =
    if n = 0
    then 1
    else if n mod 2 = 0
        then (exp_rapide a (n/2) )  * (exp_rapide a (n/2) ) 
        else (a * exp_rapide a ((n-1)/2) ) (* * (a * exp_rapide a ((n-1) /2) ) *)

val exp_rapide : int -> int -> int = <fun>


In [108]:
exp_rapide 2 3

- : int = 4


# Accumulateur

On a vu dans le cours sur la récursivité (avec l'exemple de la suite de Fibonacci) qu'un accumulateur est un argument que l'on ajoute à une fonction pour calculer sa valeur de retour.  
1. Écrire une fonction `fact` telle que `fact acc n` renvoie `n`!, en utilisant `acc` comme accumulateur. Voici à quoi va ressemble `fact` :
```ocaml
let rec fact acc n =
    if n = 0 then acc  (* on renvoie l'accumulateur qui contient le résultat *)
    else fact ... (* appel récursif en modifiant l'accumulateur *)
```
2. En utilisant `fact` et l'application partielle de fonction, définir `f : int -> int` renvoyant la factoielle d'un entier.

**Remarque** : le but ici est juste de vous entraîner à savoir utiliser un accumulateur, qui sont parfois utiles (comme pour la fonction `fibo` du cours). En DS ou concours on évitera d'utiliser un accumulateur lorsqu'il y en a pas besoin (comme pour la fonction `fact`...), car cela rend le code plus compliqué.  

In [6]:
let rec fact acc n=
    if n = 0 then acc
    else fact (acc*n) (n-1);


val fact : int -> int -> int = <fun>


In [8]:
fact 1 8

- : int = 40320


In [10]:
8*7*6*5*4*3*2

- : int = 40320


In [11]:
let fact1 n= fact 1 n

val fact1 : int -> int = <fun>


In [12]:
fact1 8

- : int = 40320


## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [28]:
let rec temps_vol a=
    if a =1 then 0 
    else
        if a mod 2 =0 then (temps_vol (a/2))+1
        else (temps_vol((3*a)+1))+1;

val temps_vol : int -> int = <fun>


In [33]:
temps_vol  15

- : int = 17


## Fonction mystérieuse

Tester la fonction suivante, conjecturer un théorème et prouvez-le par récurrence.

In [34]:
 let rec f n =
    if n > 100 then n - 10 
    else f (f (n + 11)) ;;

val f : int -> int = <fun>


In [35]:
f 1

- : int = 91


In [39]:
f 101

- : int = 91


 si n > 100-> n-10
sinon n -> n+11
donc on émet la conjecture:Pn:" pour tout n<= 101,f(n)=91"
f(101)=91
comme n=101 est vraie,on suppose que Pn est vraie pour tout n <= 100
f(100)=f(111)=91
donc,sur un intervalle n € [90,101[
]
Pn reste vraie.
Or,si Pn est vraie pour,cet intervalles,elle l'est aussi pour n€ [80,91[
]
car on a n<100 et que l'on fait n=n+11.si la propriété est vraie pour ces deux intervalles,elle est alors vraie pour tout les autres intervalles inférieur à 101.Donc pour n<=101,Pn est vraie

## Fonctions mutuellement récursives

Il est possible de définir simultanément deux fonctions `f` et `g`, dépendant l'une de l'autre avec `and` :
```ocaml
let rec f x = ... and g y = ... in ...
```

Écrire deux fonctions `u` et `v` permettant de calculer le $nième$ terme des suites définies par :
$$u_0 = 2$$
$$v_0 = 3$$
$$u_{n + 1} = u_n - u_n v_n$$
$$v_{n + 1} = v_n + u_n v_n$$

In [ ]:
let rec u n=if n=0. then 2. 
    else u (n-.1.) -.(u (n-.1.) *. v (n-.1.))
    and v n = if n=0. then 3. 
    else v(n-.1.)+.(u (n-.1.) *. v (n-.1.)) 

In [ ]:
u 1.

In [11]:
v 1.

- : float = 9.


In [13]:
u 2.

- : float = 32.


In [14]:
v 2.

- : float = -27.


In [ ]:
u 1.

## Retour sur les tours de Hanoi

**Remarque** : le problème des tours de Hanoi a déjà été vu en stage, mais on le refait ici en OCaml. En outre, il est utile de réactiver sa mémoire pour se souvenir des méthodes et il m'arrivera pendant l'année de redonner des exercices déjà posés.

![](../img/hanoi.png)

$n$ disques sont posés sur la tige à gauche. L'objectif est de déplacer tous les disques sur la tige à droite :

![](../img/hanoi2.png)

Règles du jeu :

- On ne peut déplacer qu'un disque à la fois (celui tout en haut), sur une autre tige.
- Il est interdit de poser un disque sur un autre plus petit.

Exemple de premier déplacement valide :

![](../img/hanoi3.png)

On souhaite écrire une fonction récursive `hanoi` telle que `hanoi n tige1 tige2` affiche une suite de déplacements (avec des `print_int`) permettant de déplacer $n$ disques depuis `tige1` vers `tige2`. On supposera que les tiges sont numérotées 0, 1, 2 (de gauche à droite).

1. Supposons que vous sachiez déplacer $n-1$ disques d'une tige à une autre. Comment déplacer $n$ disques d'une tige à une autre ?
2. Écrire `hanoi`.

1.si on sait déplacer n-1 disques en 1,il suffit de déplacer le dernier disque 2 er enfin de déplacer les autres disques en 2

In [22]:
let rec hanoi n tige1 tige2 tigevide= 
    if n= 1 then (Format.printf "déplacement de %d vers %d \n" tige1 tige2; print_newline())
    else (hanoi (n-1) tige1 tigevide tige2;
        (Format.printf "déplacement de %d vers %d \n" tige1 tige2);
        print_newline();
        hanoi(n-1)tigevide tige2 tige1;)
        

val hanoi : int -> int -> int -> int -> unit = <fun>


In [23]:
hanoi 3 0 2 1;


déplacement de 0 vers 2 

déplacement de 0 vers 1 

déplacement de 2 vers 1 

déplacement de 0 vers 2 

déplacement de 1 vers 0 

déplacement de 1 vers 2 

déplacement de 0 vers 2 



- : unit = ()
